1. La rete (N,W) viene inizializzata, idem per (M, R)
2. Si vettorizzano i pesi W, e li si usano come input per la rete (M, R): l'output saranno i nuovi pesi W' per la rete N. Possiamo dire quindi che la rete (M, R) computa la correzione ai pesi W della rete (N, W).
3. Usiamo i pesi W' per costruire la nuova rete (N, W')
4. Spegnamo i gradienti rispetto a tutti i pesi della rete (N, W'). Ovvero, eliminiamo tutti i pesi W' dal grafo tramite cui pytorch autodifferenzia.
5. Diamo il batch X come input a (N, W'), calcolando y e di conseguenza la loss.
6. Applichiamo il backward, che calcolerà i gradienti di tutto questo processo (iniziato al punto (2)), rispetto ai pesi R della rete (M, R), e aggiorniamo questi pesi in R'.
7. Si riparte dal punto 2, dando come input i pesi W' alla rete (M, R'), computando i nuovi pesi W'', inserendoli nella nuova rete (N, W''), e così via

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from src.models import Net, Teacher
from src.misc import vectorize_weights, get_state_dict

In [2]:
# set up the xor data
X = torch.tensor([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=torch.float)
target = torch.tensor([0, 1, 1, 0], dtype=torch.float).unsqueeze(1)

In [3]:
net = Net()

weights = vectorize_weights(net.parameters())
net_cardinality = torch.numel(weights)
print('Net cardinality: {}'.format(net_cardinality))

teacher = Teacher(net_cardinality)

Net cardinality: 9


In [4]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)
        
for e in range(1000):
    input_weights = vectorize_weights(net.parameters())

    new_weights = teacher(input_weights)
    new_state_dict = get_state_dict(net.state_dict(), new_weights) 
    net.load_state_dict(new_state_dict)
    
    pred = net(X)
    loss = criterion(pred, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [5]:
with torch.no_grad():
    print(torch.sigmoid(net(X)))
    print(target)

tensor([[0.4896],
        [0.4905],
        [0.4897],
        [0.4906]])
tensor([[0.],
        [1.],
        [1.],
        [0.]])
